In [ ]:
import numpy as np
# import laspy as lp
import pandas as pd
import rosbag # Provide utilities to process rosbag file
import rospy
import sensor_msgs.point_cloud2 as pc2 # Provide utilities to process point cloud file
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from pathlib import Path
import json

In [ ]:
config_path=Path("/home/cristina/Workspace/ProjectComputerVisionAnd3DImageProcessing/PointClouds/Configurations")
with open(config_path/"folder_config.json","r") as folder_config_json:
    folder_config=json.load(folder_config_json)

In [ ]:
data_prefix=Path(folder_config["data_prefix"])
cloudpt_bag_in=rosbag.Bag(data_prefix/folder_config["cloud_point_file"])
topic=["/cloud_map"]
print(cloudpt_bag_in.get_type_and_topic_info())
msgGen=cloudpt_bag_in.read_messages(topics=topic)
msg1=next(msgGen)
pt_cloud_msg=msg1.message

In [ ]:
cloud_position=np.array(list(pc2.read_points(pt_cloud_msg,field_names=["x","y","z","rgb"])))

In [ ]:
print(pt_cloud_msg.fields)

## Parse rgb field
- [Point Cloud2 Message Definition](http://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/PointCloud2.html)
- [Point Cloud2 Message processing apt](http://docs.ros.org/en/api/sensor_msgs/html/point__cloud2_8py_source.html#l00060)

In [ ]:
cloud_position[0]

In [ ]:
rgb_arr=cloud_position[0,-1].copy()

In [ ]:
import struct

In [ ]:
int(rgb_arr)

In [ ]:
print((struct.unpack("I",struct.pack(">f",rgb_arr))[0]>>16)&255)
print((struct.unpack("I",struct.pack(">f",rgb_arr))[0]>>8)&255)
print((struct.unpack("I",struct.pack(">f",rgb_arr))[0])&255)

In [ ]:
byte_rgb=struct.pack(">f",rgb_arr)
hex_rgb=byte_rgb.hex()

In [ ]:
print(hex_rgb) # may be the highest two hex digit not useful

In [ ]:
fmt = pc2._get_struct_fmt(pt_cloud_msg.is_bigendian, pt_cloud_msg.fields,["x","y","z","rgb"])

In [ ]:
fmt

In [ ]:
data=pt_cloud_msg.data

In [ ]:
len(data)

In [ ]:
struct.Struct(fmt).unpack_from(data,0)

In [ ]:
hex(data[0])

# Decoding Serialized data

In [ ]:
# first data "x" : datatype 7: >f, float 32. 1 bytes=32 bits
print("x, offset 0, data type 7")
print(data[0:4])
print(struct.unpack("f",data[0:4]))

# first data "y" : datatype 7: >f, float 32
print("y, offset 0, data type 7")
print(data[4:8])
print(struct.unpack("f",data[4:8]))

# first data "z" : datatype 7: >f, float 32
print("z, offset 0, data type 7")
print(data[8:12])
print(struct.unpack("f",data[8:12]))

# TODO: check how to unpack color data
# first data "rgb" : datatype 7: >f, float 32, offset 16
# "rgb"=>3 * 8bits=> 3 * 2 bytes, but here are 4 * 2 bytes, 
# which 2 bytes should be ignored?
print(data[0+16:0+16+4])
print(struct.unpack("f",data[0+16:0+16+4]))

In [ ]:
len(data)

In [ ]:
pt_cloud_msg.point_step
pt_cloud_msg.row_step

In [ ]:
len(data)/pt_cloud_msg.point_step

In [ ]:
struct.unpack(">BBBB",struct.pack(">f",rgb_arr))

In [153]:
def color_unpack(encoded_color):
    # print(encoded_color)
    data=struct.unpack(">BBBB",struct.pack(">f",encoded_color))
    return [data[1],data[2],data[3]]

# Decoding data x,y,z,r,g,b, fmt: fffxxxxBBBB

In [173]:
rgb_df=pd.DataFrame(np.apply_along_axis(color_unpack,1,cloud_position[:,-1].reshape(-1,1)),columns=["r","g","b"])
position_df=pd.DataFrame(cloud_position[:,:3],columns=["x","y","z"])
cloud_df=pd.concat([position_df,rgb_df],axis=1)

In [174]:
print(rgb_df.head())
print(len(rgb_df))
print(position_df.head())
print(len(position_df))
print(cloud_df.head())

    r   g   b
0  35  37  34
1  37  41  36
2  34  34  32
3  41  40  39
4  36  36  33
6643
          x         y         z
0  3.502988  1.214655 -1.431428
1  3.551421  1.191114 -1.432799
2  3.663276  1.207017 -1.418487
3  3.754585  1.193994 -1.413791
4  3.850423  1.212784 -1.418522
6643
          x         y         z   r   g   b
0  3.502988  1.214655 -1.431428  35  37  34
1  3.551421  1.191114 -1.432799  37  41  36
2  3.663276  1.207017 -1.418487  34  34  32
3  3.754585  1.193994 -1.413791  41  40  39
4  3.850423  1.212784 -1.418522  36  36  33
